In [1]:
import cudf.pandas
cudf.pandas.install()

import pandas as pd

In [2]:
import cupy as cp

gpus = cp.cuda.runtime.getDeviceCount()
if gpus:
    print("Available GPUs:")
    for i in range(gpus):
        print(f"GPU {i}: {cp.cuda.runtime.getDeviceProperties(i)['name']}")
else:
    print("No GPU available.")


Available GPUs:
GPU 0: b'NVIDIA GeForce RTX 3060'


In [3]:
import os
import language_tool_python
from spellchecker import SpellChecker
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import spacy
from textblob import TextBlob
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask import delayed, compute
from spellchecker import SpellChecker
import spacy
from textblob import TextBlob
import language_tool_python
import dask.dataframe as dd

In [4]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
output_file_name = 'wildberries_reviews.csv.gz'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv.gz' уже существует.


In [5]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')

Модель ru_core_news_lg уже установлена.


In [6]:
import subprocess
from language_tool_python import LanguageTool
# Запуск команды
process = subprocess.Popen(
    ['find', '/', '-name', 'languagetool-server.jar'],
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
    text=True
)

# Чтение вывода
output, _ = process.communicate()

# # Удаление пробелов и пустых строк
# LANGUAGETOOL_DIR = [line.strip() for line in output.splitlines() if line.strip()][0]

# Функция для установки необходимых пакетов и загрузки LanguageTool, если еще не загружено
def setup_languagetool():
    if not os.path.isdir(output):
        print("Установка необходимых пакетов и загрузка LanguageTool...")
        os.system('apt update && apt install -y default-jre wget unzip')
        os.system('wget https://languagetool.org/download/LanguageTool-stable.zip && unzip -o LanguageTool-stable.zip')
    else:
        print("LanguageTool уже установлен.")
        
temp = LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1})
temp.close()

In [7]:
import threading
import queue
import pandas as pd
import os
from tqdm import tqdm
import language_tool_python
from datetime import datetime

NUM_TOOLS = 6
MAX_CHECK_THREADS = 10

# Получаем текущие дату и время
current_datetime = datetime.now()

# Форматируем дату и время в российском формате с секундами
formatted_datetime = current_datetime.strftime("%d-%m-%Y_%H-%M-%S")

output_file_name = f"corrected_wildberries_reviews_{formatted_datetime}.csv"


def create_tool(tools_queue, pbar):
    tool = language_tool_python.LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1, 'maxCheckThreads': MAX_CHECK_THREADS})
    tools_queue.put(tool)
    pbar.update(1)

def correct_text(text, tool):
    if not isinstance(text, str):
        return text
    try:
        corrected_text = tool.correct(text)
        return corrected_text if corrected_text != text else text
    except Exception as e:
        print(f"Ошибка исправления текста: {e}")
        return text

def process_text(text_queue, output_file_path, save_interval, lock, progress_bar):
    temp_results = []
    while not text_queue.empty():
        idx, text, tool = text_queue.get()
        corrected_text = correct_text(text, tool)
        if corrected_text != text:
            temp_results.append((idx, corrected_text))
        text_queue.task_done()
        progress_bar.update(1)

        if len(temp_results) >= save_interval:
            with lock:
                save_partial_results(temp_results, output_file_path)
                temp_results.clear()

    if temp_results:
        with lock:
            save_partial_results(temp_results, output_file_path)

def save_partial_results(results, output_file_path):
    results.sort(key=lambda x: x[0])
    df = pd.DataFrame(results, columns=["id", 'corrected_text'])
    if os.path.exists(output_file_path):
        df.to_csv(output_file_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_file_path, index=False)
    print(f"Сохранены промежуточные результаты в {output_file_path}")

def process_data(df, output_file_path, num_tools, save_interval=1000):
    tools_queue = queue.Queue()
    threads = []
    lock = threading.Lock()

    with tqdm(total=num_tools, desc="Создание экземпляров LanguageTool") as pbar:
        for _ in range(num_tools):
            thread = threading.Thread(target=create_tool, args=(tools_queue, pbar))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

    tools = [tools_queue.get() for _ in range(num_tools)]

    text_queue = queue.Queue()

    with tqdm(total=len(df), desc="Заполнение очереди строками") as pbar:
        for idx, row in df.iterrows():
            tool = tools[idx % len(tools)]
            text_queue.put((idx, row['review_full_text'], tool))
            pbar.update(1)

    with tqdm(total=text_queue.qsize(), desc="Обработка текста") as progress_bar:
        threads = []
        for _ in range(num_tools):
            thread = threading.Thread(target=process_text, args=(text_queue, output_file_path, save_interval, lock, progress_bar))
            thread.start()
            threads.append(thread)

        text_queue.join()
        for thread in threads:
            thread.join()


df = pd.read_csv('wildberries_reviews.csv.gz', compression='gzip')[30000:34000]
process_data(df, output_file_name, num_tools=NUM_TOOLS, save_interval=100)


Обработка текста:  32%|████████████████████████████████████████████▋                                                                                                | 1269/4000 [01:10<02:10, 20.88it/s]

Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv


Обработка текста:  33%|██████████████████████████████████████████████▏                                                                                              | 1311/4000 [01:12<02:08, 20.97it/s]

Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv


Обработка текста:  33%|██████████████████████████████████████████████▌                                                                                              | 1322/4000 [01:12<01:50, 24.21it/s]

...

Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv


Обработка текста:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 3867/4000 [03:07<00:05, 25.31it/s]

Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv


Обработка текста: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [03:13<00:00, 20.63it/s]

Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv
Сохранены промежуточные результаты в corrected_wildberries_reviews_01-08-2024_10-59-53.csv


In [8]:
!ls

corrected_wildberries_reviews.csv.gz_part_1.csv.gz
corrected_wildberries_reviews_01-08-2024_10-37-24.csv.gz.csv
corrected_wildberries_reviews_01-08-2024_10-42-44.csv
corrected_wildberries_reviews_01-08-2024_10-51-00.csv
corrected_wildberries_reviews_01-08-2024_10-59-53.csv
cufile.log
rmm_log.txt
wildberries_reviews.csv
wildberries_reviews.csv.gz


In [9]:
t_df = pd.read_csv(output_file_name)
t_df

,id,corrected_text
0,30000,"Пахнет на весь салон)) пахнет супер приятно, д..."
1,30021,Пахнет каким то свежим перегаром. Явно не пако...
2,30039,Авто парфюм реально суперский🔥все ришло в цел...
3,30045,"Аромат сильный, похож на белый пакость, но как..."
4,30056,"Не знаю почему пишут то, что «не пахнет», «сов..."
...,...,...
1774,33964,Нужны кое-какие действия с домкратом . Сам клю...
1775,33970,"Цена в половину цены домкрата, что дорого как ..."
1776,33976,"Ещё не побывал, но выглядеть потрясающе, механ..."
1777,33982,Проверил работать всё хорошо 👍
